# 예제 3.51: 체크포인트(Checkpoint) 저장

## 학습목표
1. **체크포인트(Checkpoint)** 개념 이해하기 - 학습 중간 상태 저장
2. **체크포인트에 포함할 정보** 파악하기
3. **딕셔너리 형태로 여러 정보 저장** 방법 익히기
4. **학습 재개를 위한 저장 전략** 이해하기

---

#### 라이브러리 및 클래스 정의

**체크포인트(Checkpoint)란?**
- 학습 중간에 모델 상태를 저장하는 것
- 학습이 중단되어도 마지막 체크포인트부터 재개 가능
- 저장 항목: 모델 파라미터, 옵티마이저 상태, 에포크, 손실 등

In [1]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    """커스텀 데이터셋 클래스"""
    
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length


class CustomModel(nn.Module):
    """커스텀 모델 클래스"""
    
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

---

#### 데이터 및 모델 준비

In [4]:
# 데이터셋 및 데이터로더 생성
train_dataset = CustomDataset("../datasets/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)
print(train_dataset)
print(train_dataloader)



In [5]:
# GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:
# 모델, 손실함수, 옵티마이저 설정
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

---

#### 학습 및 체크포인트 저장

**체크포인트 저장 내용**
- `model`: 모델 이름 (문자열)
- `epoch`: 현재 에포크 번호
- `model_state_dict`: 모델 파라미터
- `optimizer_state_dict`: 옵티마이저 상태 (모멘텀 등)
- `cost`: 현재 손실값
- `description`: 설명 문자열

In [8]:
import os
os.makedirs("../models", exist_ok=True)

In [9]:
# 체크포인트 번호 초기화
checkpoint = 1

# 학습 루프
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    # 1000 에포크마다 체크포인트 저장
    if (epoch + 1) % 1000 == 0:
        # 딕셔너리 형태로 여러 정보를 한 번에 저장
        torch.save(
            {
                "model": "CustomModel",                      # 모델 이름
                "epoch": epoch,                              # 현재 에포크
                "model_state_dict": model.state_dict(),      # 모델 파라미터
                "optimizer_state_dict": optimizer.state_dict(),  # 옵티마이저 상태
                "cost": cost,                                # 현재 손실
                "description": f"CustomModel 체크포인트-{checkpoint}",
            },
            f"../models/checkpoint-{checkpoint}.pt",
        )
        print(f"체크포인트 {checkpoint} 저장 완료 (Epoch: {epoch+1})")
        checkpoint += 1

체크포인트 1 저장 완료 (Epoch: 1000)
체크포인트 2 저장 완료 (Epoch: 2000)
체크포인트 3 저장 완료 (Epoch: 3000)
체크포인트 4 저장 완료 (Epoch: 4000)
체크포인트 5 저장 완료 (Epoch: 5000)
체크포인트 6 저장 완료 (Epoch: 6000)
체크포인트 7 저장 완료 (Epoch: 7000)
체크포인트 8 저장 완료 (Epoch: 8000)
체크포인트 9 저장 완료 (Epoch: 9000)
체크포인트 10 저장 완료 (Epoch: 10000)


---

#### 모델 추론

In [10]:
# 추론 수행
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print("예측 결과:")
    print(outputs)

예측 결과:
tensor([[  1.4929],
        [ 69.2348],
        [357.2300]], device='cuda:0')
